In [3]:
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt

def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn

from sklearn.preprocessing import LabelEncoder

#train = pd.read_csv('../input/train.csv')
#test = pd.read_csv('../input/test.csv')

import nltk
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import sys
import re
import glob
import gzip
from lxml import etree
from nltk import sent_tokenize, word_tokenize, pos_tag, ne_chunk
import nltk.data
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
import pickle
from collections import Iterable
from nltk.tag import ClassifierBasedTagger
from nltk.chunk import ChunkParserI
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.chunk import conlltags2tree, tree2conlltags
from sklearn.metrics import accuracy_score, log_loss
import random
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
    naive_bayes.MultinomialNB(),
    KNeighborsClassifier(2),
    SVC(kernel="rbf", C=0.025, probability=True),
    NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    #GaussianNB(),
    #LinearDiscriminantAnalysis(),
    #QuadraticDiscriminantAnalysis(),
]

log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

#Local imports
%load_ext autoreload
%autoreload 2
from unwiki import unwiki
import ner
import parsing_xml as px

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
%%time
xml_lst = glob.glob("/mnt/training_defs/math1*/*.xml.gz")
#allData = pd.DataFrame()
all_data_texts = []
all_data_labels = []
def_cnt = 0
nondef_cnt = 0
for X in xml_lst[:1000]:
    tar_tree = etree.parse(X)
    def_lst = tar_tree.findall('.//definition')
    nondef_lst = tar_tree.findall('.//nondef')
    all_data_texts += [D.text for D in def_lst]
    all_data_labels += len(def_lst)*[1.0]
    all_data_texts += [D.text for D in nondef_lst]
    all_data_labels += len(nondef_lst)*[0.0]
    def_cnt += len(def_lst)
    nondef_cnt += len(nondef_lst)
print("Definition count: %s.   NonDefinitions count: %s. Total: %s"%(def_cnt, nondef_cnt, (def_cnt+nondef_cnt)))

Definition count: 330889.   NonDefinitions count: 327492. Total: 658381
CPU times: user 14.1 s, sys: 1.95 s, total: 16.1 s
Wall time: 59.6 s


In [5]:
%%time
# define Clean function to cleanse and standarize words
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalize

#prepare the dataset Using 2018 old dataset
#allData = pd.DataFrame()
#with open('../sample18/defs.txt','r') as f1:
#    all_data_texts = f1.readlines()
#all_data_labels = len(all_data_texts)*[1.0]
#with open('../sample18/nondefs.txt', 'r') as f2:
#    all_data_texts_rand = f2.readlines()
#all_data_texts += all_data_texts_rand
#all_data_labels += len(all_data_texts_rand)*[0.0]

# 1.0 will represent definitions is true 0.0 means it is false (not a definition)
#allData['labels'] = all_data_labels
#allData['texts'] = all_data_texts

# Split and randomize the datasets
train_x, test_x, train_y, test_y = model_selection.train_test_split(all_data_texts, all_data_labels)

# Vectorize all the paragraphs and definitions in the dataset
count_vect = CountVectorizer(analyzer='word', tokenizer=nltk.word_tokenize, ngram_range=(1,3))
count_vect.fit(all_data_texts)
xtrain = count_vect.transform(train_x)
xtest = count_vect.transform(test_x)

# Train Multinomial Naive Bayes model and print test metrics
#clf = naive_bayes.MultinomialNB().fit(xtrain, train_y)
#predictions = clf.predict(xtest)
#print(metrics.classification_report(predictions,test_y))

MemoryError: 

In [5]:
%%time
Def = ['a banach space is defined as a complete vector space.',
       'This is not a definition honestly. even if it includes technical words like scheme and cohomology',
      'There is no real reason as to why this classifier is so good.',
      'a triangle is equilateral if and only if all its sides are the same length.',
      ' The paper is organized as follows. ',
      'Proof. By definition (6.4) _display_math_ where _inline_math_ denotes the parity of _inline_math_ and _display_math_',
      'Counting subobjects over finite fields, as in Ringel _citation_.']
vdef = count_vect.transform(Def)
clf.predict(vdef)

AttributeError: 'SVC' object has no attribute '_probA'

In [3]:
with open('/mnt/PickleJar/count_vectorizer49.pickle', 'rb') as pickle_obj:
    count_vect = pickle.load(pickle_obj)
with open('/mnt/PickleJar/classifier49.pickle', 'rb') as pickle_obj:
    clf = pickle.load(pickle_obj)

In [5]:
#param_lst = [1550, 1650, 1750]
#SVC_lst = [SVC(kernel="rbf", C=param, probability=True) for param in param_lst]
#classifiers=  [ naive_bayes.MultinomialNB(),]
classifiers=  [SVC(kernel="rbf", C=1600, probability=True)]

In [31]:
#for C_param, clf in zip(param_lst,SVC_lst):
for C_param, clf in enumerate(classifiers):
    name = clf.__class__.__name__
    clf.fit(xtrain, train_y)
    
    print("="*30)
    print(name, ',  C=', C_param)
    
    print('****Results****')
    predictions = clf.predict(xtest)
    acc = accuracy_score(test_y, predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    #prodictions = prob_prediction
    prodictions = clf.predict_proba(xtest)
    ll = log_loss(test_y, prodictions)
    print("Log Loss: {}".format(ll))
    
    #log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    #log = log.append(log_entry)
    
print("="*30)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-31-97796aa0bddd>", line 4, in <module>
    clf.fit(xtrain, train_y)
  File "/usr/lib/python3.8/site-packages/sklearn/svm/_base.py", line 177, in fit
    X_var = ((X.multiply(X)).mean() - (X.mean()) ** 2
  File "/usr/lib/python3.8/site-packages/scipy/sparse/base.py", line 1082, in mean
    return (inter_self / np.array(
  File "/usr/lib/python3.8/site-packages/scipy/sparse/base.py", line 617, in __truediv__
    return self._divide(other, true_divide=True)
  File "/usr/lib/python3.8/site-packages/scipy/sparse/base.py", line 582, in _divide
    return self.astype(np.float_)._mul_scalar(1./other)
  File "/usr/lib/python3.8/site-packages/scipy/sparse/data.py", line 74, in astype
    return self.copy()
  File "/usr/lib/python3.8/site-packages/scipy/sparse/data.py", line 91, in copy
  

KeyboardInterrupt: 

In [4]:
print(metrics.classification_report(predictions,test_y))

NameError: name 'predictions' is not defined

In [37]:
ex_nondef = [D.text for D in nondef_lst[:15]]
clf.predict(count_vect.transform(ex_nondef))

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [35]:
tar_tree = etree.parse('/mnt/training_defs/math10/1002_005.xml.gz')
def_lst = tar_tree.findall('.//definition')
nondef_lst = tar_tree.findall('.//nondef')
ex_text = [D.text for D in nondef_lst[:100]]
sum(clf.predict(count_vect.transform(ex_text)))

16.0

In [ ]:
with open('../PickleJar/count_vectorizer49.pickle', 'wb') as class_f:
    pickle.dump(count_vect, class_f)

In [38]:
len(ww)

4939532

In [13]:
# Get the n-grams
ww = count_vect.get_feature_names()

In [36]:
# Get the n-grams (same as features) with identity number (not freq count)
count_vect.vocabulary_

['like the multivariate',
 'the multivariate autoregressive',
 'multivariate autoregressive process',
 'autoregressive process that',
 '3. for each',
 'sum _display_math_ throughout',
 'absolute values less',
 'stationary distribution which',
 'normal with the',
 'zero vector as',
 'vector as mean',
 'as mean and',
 'the original univariate',
 'original univariate process',
 'univariate process _inline_math_',
 'the element at',
 'measures induced',
 'probability measures induced',
 'measures induced by',
 'rate of speed',
 'of speed and',
 'speed and the',
 'folner',
 '_inline_math_-maximality',
 'some amenable',
 'weak folner',
 'folner sequence',
 'an _inline_math_-maximality',
 '_inline_math_-maximality condition',
 'be some amenable',
 'some amenable group',
 'group , along',
 'a weak folner',
 'weak folner sequence',
 'folner sequence _inline_math_',
 'be some bounded',
 'bounded , additive',
 ', additive process',
 'additive process as',
 'satisfies an _inline_math_-maximality',

In [21]:
s = 2000000
l = 100
ww[s:(s+l)]

['fibre _inline_math_ and',
 'fibre _inline_math_ are',
 'fibre _inline_math_ as',
 'fibre _inline_math_ associated',
 'fibre _inline_math_ au',
 'fibre _inline_math_ belongs',
 'fibre _inline_math_ can',
 'fibre _inline_math_ denotes',
 'fibre _inline_math_ for',
 'fibre _inline_math_ has',
 'fibre _inline_math_ have',
 'fibre _inline_math_ hence',
 'fibre _inline_math_ in',
 'fibre _inline_math_ intersects',
 'fibre _inline_math_ into',
 'fibre _inline_math_ is',
 'fibre _inline_math_ itself',
 'fibre _inline_math_ mais',
 'fibre _inline_math_ modulo',
 'fibre _inline_math_ must',
 'fibre _inline_math_ notice',
 'fibre _inline_math_ of',
 'fibre _inline_math_ on',
 'fibre _inline_math_ over',
 'fibre _inline_math_ où',
 'fibre _inline_math_ pour',
 'fibre _inline_math_ smooth',
 'fibre _inline_math_ smoothly',
 'fibre _inline_math_ such',
 'fibre _inline_math_ then',
 'fibre _inline_math_ where',
 'fibre _inline_math_ which',
 'fibre _inline_math_ with',
 'fibre _inline_math_-plectic